In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy
nltk.download('punkt')

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]
NLTK: 3.4.4
Scikit-learn: 0.21.3
Pandas: 0.25.0
Numpy: 1.17.0


[nltk_data] Downloading package punkt to /home/audora/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Carregando dados
import pandas as pd

df = pd.read_csv('../data/dataset_multi.txt', header=None, encoding='utf-8', sep='\t')
df.fillna('x', inplace=True)
#df_classes_raw = pd.read_csv('data/classes_raw.txt', header=None, encoding='utf-8', sep='\t')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null object
1    793 non-null object
dtypes: object(2)
memory usage: 12.5+ KB
None
                0                                                  1
0          outros  pretensa estadual deduz apel extrem esbarr ved...
1           civil  embarg insist mesm razo recurs apresent oposic...
2           civil  acorda ora recorr nao neg vigenc direit federa...
3           civil  tribunal justic assent compreensa part det leg...
4  administrativo  segund jurisprudenc superior tribunal justic a...


In [8]:
# verificação das classes dos acórdãos

classes = df[0]
print(len(classes.value_counts()))
classes.value_counts().to_frame().style.bar()

8


,0
civil,271
outros,198
administrativo,103
penal,75
tributário,64
constitucional,61
previdenciário,20
empresarial,1


In [9]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes
encoder = LabelEncoder()
classes = encoder.fit_transform(classes)

print(classes[:10])
len(classes)

[4 1 1 1 0 2 5 5 4 4]


793

In [10]:
# criação da lista de ementas

ementas = df[1]
print(ementas[:10])

0    pretensa estadual deduz apel extrem esbarr ved...
1    embarg insist mesm razo recurs apresent oposic...
2    acorda ora recorr nao neg vigenc direit federa...
3    tribunal justic assent compreensa part det leg...
4    segund jurisprudenc superior tribunal justic a...
5    sobrest recurs especial ate julgament recurs e...
6    inadmissivel exigenc recolh reu prisa requisit...
7    dentr limit leg vez caracteriz reincidenc agra...
8    sobr prescrica aca repetica indebit tributari ...
9    tratas agrav instrument interpost contr decisa...
Name: 1, dtype: object


In [11]:
# uso de word2vec para a extração de features

from gensim.models import Word2Vec

ementas_list = [word for word in ementas.iteritems()]
ementas_list_list = []
for doc in ementas_list:
    ementas_list_list.append(doc[1].split())

X = Word2Vec(ementas_list_list, min_count=2)
X.wv.init_sims() # para inicializar model.vw.syn0norm, necessário para o cálculo da média da
print(X)

Word2Vec(vocab=3183, size=100, alpha=0.025)


In [15]:
# funções para calcular a média vetorial das palavras de cada ementa
import numpy as np

def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        print("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [16]:
# função para tokenizar as palavras com frequência maior que 2
import nltk


def w2v_tokenize_text(text):    
    tokens = []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [21]:
# Like any other supervised machine learning problem, we need to divide data into 
# 20% test set and 80% training set

import gensim
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


train, test = train_test_split(df, test_size=0.2, random_state=0)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values

X_train = word_averaging_list(X.wv,train_tokenized)
X_test = word_averaging_list(X.wv,test_tokenized)

y_train = train[0]
y_test = test[0]

print(X_train)
print(X_test)

cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
[[ 0.05816281  0.06925008  0.01058854 ... -0.05692833 -0.01062296
  -0.01773864]
 [ 0.05668664  0.0714181   0.01187222 ... -0.05546273 -0.01052836
  -0.00976634]
 [ 0.05447049  0.07175037  0.00836629 ... -0.05591553 -0

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 30.18867924528302
Decision Tree Accuracy: 32.70440251572327
Random Forest Accuracy: 37.10691823899371
Logistic Regression Accuracy: 41.509433962264154
SGD Classifier Accuracy: 17.61006289308176
Naive Bayes Accuracy: 41.509433962264154


In [23]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_train, y_train)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 35.84905660377358


In [24]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print((accuracy_score(y_test, y_pred)*100))

                precision    recall  f1-score   support

administrativo       0.22      0.08      0.12        24
         civil       0.43      0.81      0.56        63
constitucional       0.00      0.00      0.00        12
        outros       0.15      0.16      0.15        25
         penal       0.00      0.00      0.00        14
previdenciário       0.00      0.00      0.00         7
    tributário       0.00      0.00      0.00        14

      accuracy                           0.36       159
     macro avg       0.11      0.15      0.12       159
  weighted avg       0.23      0.36      0.26       159

35.84905660377358


/home/audora/tcc/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:
# usando undersampling
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy='majority', random_state=0)
rus.fit(X_train, y_train)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
print("train: {},{}".format(len(X_resampled),len(y_resampled)))


train: 427,427


In [32]:
# Fazendo a mesma análisa usando undersampling

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_resampled, y_resampled)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 15.09433962264151
Decision Tree Accuracy: 18.238993710691823
Random Forest Accuracy: 20.125786163522015
Logistic Regression Accuracy: 15.723270440251572
SGD Classifier Accuracy: 15.09433962264151
Naive Bayes Accuracy: 15.723270440251572


In [34]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_resampled, y_resampled)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 15.723270440251572


In [35]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print((accuracy_score(y_test, y_pred)*100))

                precision    recall  f1-score   support

administrativo       0.10      0.04      0.06        24
         civil       0.00      0.00      0.00        63
constitucional       0.00      0.00      0.00        12
        outros       0.16      0.96      0.28        25
         penal       0.00      0.00      0.00        14
previdenciário       0.00      0.00      0.00         7
    tributário       0.00      0.00      0.00        14

      accuracy                           0.16       159
     macro avg       0.04      0.14      0.05       159
  weighted avg       0.04      0.16      0.05       159

15.723270440251572


/home/audora/tcc/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
